In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
def get_sortinghat_type(col):
    """ OpenML's conversion from Pandas dtypes to ARFF format
    """
    PD_DTYPES_TO_ARFF_DTYPE = {"integer": "INTEGER", "floating": "REAL", "string": "STRING"}
    attributes_arff = []

    # skipna=True does not infer properly the dtype. The NA values are
    # dropped before the inference instead.
    column_dtype = pd.api.types.infer_dtype(col.dropna(), skipna=False)
    
    if column_dtype == "categorical":
        return 'categorical'
    elif column_dtype == "boolean":
        return 'categorical'
    elif column_dtype in ('integer', 'floating'):
        return 'numeric'
    # elif column_dtype == 'string':
    #     return 'sentence'
    
    return 'context-specific'

In [12]:
testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

# print(test_merged)
y_true = test_merged.y_act.values.tolist()
# print(y_true)
print(len(y_true))

1985 4233
1985
1985


In [4]:
from tqdm.notebook import tqdm

y_pandas = []
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in tqdm(test_merged.iterrows(), total=len(test_merged)):
# #     print(index)    
#     if index%100==0: 
# #         print(y_tfdv)
#         print(index)
    col = row['Attribute_name']
    prv_csv_name = csv_name
    csv_name = '../RawCSVFiles/' + row['name']
    
    # print(csv_name)
    # print(col)

    if prv_csv_name != csv_name:
        df = pd.read_csv(csv_name,encoding='latin1')
    
    try:
        curtype = get_sortinghat_type(df[col])
    except KeyError:
        curtype = 'Object'
    # print(curtype)
    y_pandas.append(curtype)

/home/bobotran/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (41,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/bobotran/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (40,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/bobotran/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (403,405,832,833,841,844,857,915,916,928,929,941,942,954,955,957,958,967,968,970,971,980,981,993,994,996,997,1006,1007,1009,1010,1022,1023,1035,1036,1048,1049,1071,1072,1084,1085,1087,1088,1097,1098,1100,1101,1110,1111,1113,1114,1126,1127,1139,1140,1149,1150,1152,1153,1162,1163,1165,1166,1178,1179,1191,1192,1204,1205,1217,1218,1227,1228,1230,1231,1240,1241,1243,1244,1256

In [6]:
print(len(y_pandas))
print(y_pandas[:20])

1985
['numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric']


In [7]:
class_map = {
    0: 'numeric',
    1: 'categorical',
    2: 'datetime',
    3: 'sentence',
    4: 'url',
    5: 'embedded-number',
    6: 'list',
    7: 'not-generalizable',
    8: 'context-specific'
}

In [8]:
from sklearn.metrics import classification_report
import numpy as np

print('Accuracies: ')
print(accuracy_score(y_true, y_pandas))
for v in class_map.values():
    one_vs_all_acc = accuracy_score(np.array(y_true) == v, np.array(y_pandas) == v)
    print('\t{}: {}'.format(v, one_vs_all_acc))
matrix = confusion_matrix(y_true, y_pandas, labels=list(class_map.values()))
print(matrix)
print(classification_report(y_true, y_pandas, labels=list(class_map.values()), digits=3))

Accuracies: 
0.383375314861461
	numeric: 0.7768261964735517
	categorical: 0.7712846347607053
	datetime: 0.9289672544080605
	sentence: 0.9536523929471032
	url: 0.9838790931989925
	embedded-number: 0.9501259445843828
	list: 0.9712846347607053
	not-generalizable: 0.8916876574307305
	context-specific: 0.5476070528967254
[[706   0   0   0   0   0   0   0   1]
 [178   3   0   0   0   0   0   0 275]
 [  4   0   0   0   0   0   0   0 137]
 [  0   0   0   0   0   0   0   0  92]
 [  0   0   0   0   0   0   0   0  32]
 [  0   0   0   0   0   0   0   0  85]
 [  0   0   0   0   0   0   0   0  57]
 [127   0   0   0   0   0   0   0  86]
 [133   0   0   0   0   0   0   0  52]]
                   precision    recall  f1-score   support

          numeric      0.615     0.999     0.761       707
      categorical      1.000     0.007     0.013       457
         datetime      0.000     0.000     0.000       141
         sentence      0.000     0.000     0.000        92
              url      0.000     0

/home/bobotran/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
[(i, yt, yp) for (i, yt, yp) in zip(range(len(testdf)), y_true, y_pandas) if yt == 'categorical' and yp == 'numeric']

[(3, 'categorical', 'numeric'),
 (17, 'categorical', 'numeric'),
 (18, 'categorical', 'numeric'),
 (27, 'categorical', 'numeric'),
 (28, 'categorical', 'numeric'),
 (29, 'categorical', 'numeric'),
 (30, 'categorical', 'numeric'),
 (31, 'categorical', 'numeric'),
 (32, 'categorical', 'numeric'),
 (38, 'categorical', 'numeric'),
 (60, 'categorical', 'numeric'),
 (61, 'categorical', 'numeric'),
 (62, 'categorical', 'numeric'),
 (63, 'categorical', 'numeric'),
 (71, 'categorical', 'numeric'),
 (72, 'categorical', 'numeric'),
 (73, 'categorical', 'numeric'),
 (74, 'categorical', 'numeric'),
 (126, 'categorical', 'numeric'),
 (128, 'categorical', 'numeric'),
 (136, 'categorical', 'numeric'),
 (137, 'categorical', 'numeric'),
 (139, 'categorical', 'numeric'),
 (140, 'categorical', 'numeric'),
 (179, 'categorical', 'numeric'),
 (180, 'categorical', 'numeric'),
 (181, 'categorical', 'numeric'),
 (182, 'categorical', 'numeric'),
 (183, 'categorical', 'numeric'),
 (184, 'categorical', 'numeric'),

In [13]:
test_merged.iloc[3]

Record_id                                                      285
Attribute_name                                            For Year
y_act                                                  categorical
total_vals                                                    1781
num_nans                                                       173
%_nans                                                     9.71364
num_of_dist_val                                                  6
%_dist_val                                                0.336889
mean                                                       2013.31
std_dev                                                    19.9469
min_val                                                       1215
max_val                                                       2016
sample_1                                                      2012
sample_2                                                      2013
sample_3                                                      